In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import os
import random
import tensorflow as tf
import cv2
import numpy as np
import time
import csv
import glob
import shutil
import sys
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Thepath = "/Users/george/Desktop/ck+/Images"
anpath = "/Users/george/Desktop/ck+/Images/anger"
conpath = "/Users/george/Desktop/ck+/Images/contempt"
dispath = "/Users/george/Desktop/ck+/Images/disgust"
fpath = "/Users/george/Desktop/ck+/Images/fear"
hapath = "/Users/george/Desktop/ck+/Images/happy"
sapath = "/Users/george/Desktop/ck+/Images/sadness"
supath = "/Users/george/Desktop/ck+/Images/surprise"
Worldpath = "/Users/george/Desktop/E.IMAGES/THEWORLD"

path = "/Users"
image_dir = os.path.join(path, "george", "Desktop", "ck+", "cohn-kanade-images")
emotion_dir = os.path.join(path, "george", "Desktop", "ck+", "Emotion")

def get_image_and_labels(data_dir, all=True):
    path = "/Users"
    image_dir = os.path.join(path, "george", "Desktop", "ck+", "cohn-kanade-images")
    emotion_dir = os.path.join(path, "george", "Desktop", "ck+", "Emotion")
    
    image = []
    label = []
    
    for root, dirs, files in os.walk(emotion_dir):
        file_len = len(files)
        if file_len != 0:
            for file in files: #file S005_001_00000011_emotion.txt
                basefile, ext = os.path.splitext(file)
                if ext == '.txt':
                    base = basefile.rsplit('_', 1)[0] #base S005_001_00000011
                    basedirs = base.split('_') #basedirs[0] S005 #basedirs[1] 001 #basedirs[2] 00000011
                    image_num = int(basedirs[2])
                    emotion_file = os.path.join(emotion_dir, basedirs[0], basedirs[1], file)
                    
                    f = open(emotion_file, 'r')
                    for line in f:
                        line = line.strip()
                        line = int(float(line))
                        line = line - 1
                        

                    if all == True:
                        temp_imagedir = os.path.join(image_dir, basedirs[0], basedirs[1])
                        for root, dirs, files in os.walk(temp_imagedir):
                            for file in files:
                                bfile, ext = os.path.splitext(file)
                                if ext == '.png':
                                    image.append(os.path.join(temp_imagedir, file))
                                    label.append(line)
                    else:
                        temp_imagedir_file = os.path.join(image_dir, basedirs[0], basedirs[1], "%s.png" % base)
                        image.append(temp_imagedir_file)
                        label.append(line)
                
             
    return image, label

def creating_the_folders(classes):
    for i in range(7):
        folders = os.path.join(Thepath, classes[i])
        os.makedirs(folders)
                
def filling_the_folders(images,labels):
    for i in range(0,len(images)):
        for jpgfile in glob.iglob(os.path.join(images[i])):
            if labels[i] == 0:
                shutil.copy(jpgfile, anpath)
            elif labels[i] == 1:
                shutil.copy(jpgfile, conpath)
            elif labels[i] == 2:
                shutil.copy(jpgfile, dispath)
            elif labels[i] == 3:
                shutil.copy(jpgfile, fpath)
            elif labels[i] == 4:
                shutil.copy(jpgfile, hapath)
            elif labels[i] == 5:
                shutil.copy(jpgfile, sapath)
            elif labels[i] == 6:
                shutil.copy(jpgfile, supath)
                                
# Parameters
LEARNING_RATE = 0.01
ITERATIONS = 50
BATCH_SIZE = 64
DISPLAY_STEP = 1
CLASSES = 7
CHANNELS = 1
IMAGE_SIZE = 96
N_INPUT = IMAGE_SIZE * IMAGE_SIZE
DROPOUT = 0.5

      
train_dir = os.path.join(Thepath, 'train')
val_dir = os.path.join(Thepath, 'val')

image_gen_train = ImageDataGenerator(rescale=1./255, 
                                     featurewise_center=False,
                                     featurewise_std_normalization=False,
                                     rotation_range=10,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     zoom_range=.1,
                                     horizontal_flip=True)

train_data_gen = image_gen_train.flow_from_directory(
                                                batch_size=BATCH_SIZE,
                                                directory=train_dir,
                                                shuffle=True,
                                                target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                class_mode='sparse'
                                                )

image_gen_val = ImageDataGenerator(rescale=1./255) 

val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=val_dir,
                                                 target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                 class_mode='sparse')

model = Sequential()

model.add(Conv2D(64, 5, padding='valid', bias_initializer="zeros", input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, 5, padding='valid', bias_initializer="zeros"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, 5, padding='valid', bias_initializer="zeros"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))

model.add(Dense(7, activation='softmax'))

sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True)

model.compile( loss = "sparse_categorical_crossentropy", 
               optimizer = sgd, 
               metrics=['accuracy']
             )            

epochs = 300

history = model.fit(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)

model.save('Masks')

Found 260 images belonging to 7 classes.
Found 67 images belonging to 7 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 5 steps, validate for 2 steps
Epoch 1/200
5/5 [==============================] - 6s 1s/step - loss: 1.8950 - accuracy: 0.2192 - val_loss: 1.8160 - val_accuracy: 0.2537
Epoch 2/200
5/5 [==============================] - 2s 311ms/step - loss: 1.9024 - accuracy: 0.2269 - val_loss: 1.8372 - val_accuracy: 0.2090
Epoch 3/200
5/5 [==============================] - 2s 313ms/step - loss: 1.8572 - accuracy: 0.2115 - val_loss: 1.8145 - val_accuracy: 0.2090
Epoch 4/200
5/5 [==============================] - 2s 312ms/step - loss: 1.8583 - accuracy: 0.2192 - val_loss: 1.7945 - val_accuracy: 0.2090
Epoch 5/200
5/5 [==============================] - 2s 321ms/step - loss: 1.8473 - accuracy: 0.2154 - val_loss: 1.7908 - val_accuracy: 0.2090
Epoch 6/200
5/5 [==============================] - 2s 312ms/step - loss: 1.8393 - accuracy: 0.2154 - val_loss: 1.7975 - val_acc

Epoch 57/200
5/5 [==============================] - 2s 304ms/step - loss: 1.2904 - accuracy: 0.4538 - val_loss: 1.0743 - val_accuracy: 0.5075
Epoch 58/200
5/5 [==============================] - 1s 299ms/step - loss: 1.2193 - accuracy: 0.4885 - val_loss: 0.9501 - val_accuracy: 0.5522
Epoch 59/200
5/5 [==============================] - 2s 315ms/step - loss: 1.2437 - accuracy: 0.4692 - val_loss: 0.9898 - val_accuracy: 0.6119
Epoch 60/200
5/5 [==============================] - 2s 318ms/step - loss: 1.2023 - accuracy: 0.5115 - val_loss: 0.9106 - val_accuracy: 0.6567
Epoch 61/200
5/5 [==============================] - 2s 311ms/step - loss: 1.1402 - accuracy: 0.5154 - val_loss: 0.9489 - val_accuracy: 0.5821
Epoch 62/200
5/5 [==============================] - 2s 304ms/step - loss: 1.1050 - accuracy: 0.5500 - val_loss: 0.8550 - val_accuracy: 0.5672
Epoch 63/200
5/5 [==============================] - 2s 313ms/step - loss: 1.1465 - accuracy: 0.5154 - val_loss: 0.9081 - val_accuracy: 0.5672
Epoch 

5/5 [==============================] - 2s 310ms/step - loss: 0.1416 - accuracy: 0.9462 - val_loss: 0.1693 - val_accuracy: 0.8657
Epoch 172/200
5/5 [==============================] - 2s 312ms/step - loss: 0.0988 - accuracy: 0.9654 - val_loss: 0.1636 - val_accuracy: 0.8507
Epoch 173/200
5/5 [==============================] - 2s 309ms/step - loss: 0.1305 - accuracy: 0.9500 - val_loss: 0.1961 - val_accuracy: 0.8507
Epoch 174/200
5/5 [==============================] - 2s 305ms/step - loss: 0.1397 - accuracy: 0.9500 - val_loss: 0.2306 - val_accuracy: 0.8209
Epoch 175/200
5/5 [==============================] - 2s 314ms/step - loss: 0.1236 - accuracy: 0.9423 - val_loss: 0.1892 - val_accuracy: 0.8806
Epoch 176/200
5/5 [==============================] - 2s 312ms/step - loss: 0.1588 - accuracy: 0.9462 - val_loss: 0.1807 - val_accuracy: 0.8507
Epoch 177/200
5/5 [==============================] - 2s 309ms/step - loss: 0.1207 - accuracy: 0.9462 - val_loss: 0.1677 - val_accuracy: 0.8806
Epoch 178/200